## Initialize the environment

In [ ]:
pip install networkx

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col 
import networkx as nx
import pandas as pd
from itertools import islice
import itertools
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets
import pandas as pd

pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.max_rows', None)  # Display all rows
pd.set_option('display.max_colwidth', None)  # Disable truncation of column contents

spark = SparkSession.builder \
    .appName("Router") \
    .master("local") \
    .getOrCreate()

## Loading edges

In [ ]:
complete_edges = spark.read.orc('/group/grande_envergure/graph/complete_edges.orc')
df_all_edges= complete_edges.toPandas()
df_all_edges.head()

In [ ]:
def get_sec(time_str):
    """Get seconds from time.

    Args:
        time_str: A string representing time in the format "HH:MM:SS".

    Returns:
        seconds: The total number of seconds represented by the input time.
    """
    h, m, s = time_str.split(':')
    return int(h) * 3600 + int(m) * 60 + int(s)


def get_time(seconds):
    """Get time from seconds.

    Args:
        seconds: An integer representing the total number of seconds.

    Returns:
        time_str: A string representing the time in the format "HH:MM:SS".
    """
    m, s = divmod(seconds, 60)
    h, m = divmod(m, 60)
    return "%02d:%02d:%02d" % (h, m, s)

Transforming the times into seconds and keeping only valid edges 

In [ ]:
df_all_edges["start_time"]=df_all_edges["start_time"].apply(lambda x: None if x is None else get_sec(x))
df_all_edges["end_time"]=df_all_edges["end_time"].apply(lambda x: None if x is None else get_sec(x))
df_all_edges= df_all_edges[~df_all_edges["expected_travel_time"].isnull()]

## Loading nodes

In [ ]:
nodes_zurich = spark.read.orc('/group/grande_envergure/graph/nodes_zurich.orc')
nodes_zurich= nodes_zurich.toPandas()
nodes_zurich.head()

# Finding the best Paths

First let us define some usefull functions to find the best paths 

In [ ]:
def add_path_info(path_graph):
    """Compute the infos about the points for the shortest paths in the graph.
    Args:
        path_graph: A NetworkX DiGraph instance.
    Returns:
        paths_info: A list of lists of dictionaries containing information about each point in the paths.
    """
    
    # Calculate all shortest paths
    all_shortest_paths = nx.all_shortest_paths(path_graph, "start", "end", weight="expected_travel_time")

    # Get the first 20 paths
    first_20_shortest_paths = list(itertools.islice(all_shortest_paths, 20))
    # Initialise the infos array 
    paths_info = []
    
    # Traverse each path
    for path in first_20_shortest_paths:
        path_info = []
        # Traverse the nodes in the path
        for node in path[1:-1]:
            # Add edges infos 
            src_departure_time = path_graph.nodes[node]["start_time"]
            dst_arrival_time = path_graph.nodes[node]["end_time"]
            src = path_graph.nodes[node]["start_stop_id"]
            dst = path_graph.nodes[node]["end_stop_id"]
            trip = path_graph.nodes[node]["trip_id"]
            edge = {"start_time":get_time(src_departure_time), 
                 "end_time":get_time(dst_arrival_time), 
                 "start_stop_id":src, 
                 "end_stop_id":dst, 
                 "trip_id":trip}
            path_info.append(edge)
        
        paths_info.append(path_info)

    return paths_info

def add_node_to_graph(graph, index, row):
    """Add a node to the given graph.

    Args:
        graph (networkx.DiGraph): The graph to which the node will be added.
        index (int): The identifier for the node.
        row (pandas.Series): The row of data containing node attributes.

    Returns:
        None
    """
    graph.add_node(
        index, 
        start_stop_id=row['start_stop_id'], 
        end_stop_id=row['end_stop_id'], 
        start_time=row['start_time'], 
        end_time=row['end_time'], 
        expected_travel_time=row['expected_travel_time'],
        trip_id=row['trip_id']
    )

def add_walking_nodes_to_graph(path_graph, start, end, extracted, edges, row, i):
    """Add walking nodes to the given graph.

    Args:
        path_graph (networkx.DiGraph): The graph to which the nodes will be added.
        start (str): The identifier for the start node.
        end (str): The identifier for the end node.
        extracted (list): The extracted nodes.
        edges (pandas.DataFrame): The dataframe containing the edge information.
        row (pandas.Series): The row of data containing node attributes.
        i (int): The incremental index for the node.

    Returns:
        i (int): The updated incremental index for the node.
    """
    if start == extracted[0][0]:
        connections_next = edges[(edges["start_stop_id"]==end) & (edges["is_walking"]==0)]
        for index_next, row_next in connections_next.iterrows():
            walk_time = row["expected_travel_time"]
            walk_time_end = row_next["start_time"]
            walk_time_start = walk_time_end - walk_time
            node_attributes = {
                "start_time": walk_time_start,
                "end_time": walk_time_end,
                "expected_travel_time": walk_time,
                "start_stop_id": row["start_stop_id"],
                "end_stop_id": row["end_stop_id"],
                "trip_id": "None"
            }
            path_graph.add_node(index_next + i, **node_attributes)
            i += 1
    else:
        connections_prev = [(node, attr) for node, attr in path_graph.nodes(data=True) if attr["end_stop_id"]==row["start_stop_id"]]
        for prev_node, prev_attr in connections_prev:
            walk_time = row["expected_travel_time"]
            walk_time_start = prev_attr["end_time"]
            walk_time_end = walk_time_start + walk_time
            node_attributes = {
                "start_time": walk_time_start,
                "end_time": walk_time_end,
                "expected_travel_time": walk_time,
                "start_stop_id": row["start_stop_id"],
                "end_stop_id": row["end_stop_id"],
                "trip_id": "None"
            }
            path_graph.add_node(prev_node + i, **node_attributes)
            i += 1

    return i

In [ ]:
def get_path(extracted, edges):
    """Given a dataframe of edges and pairs of nodes return feasable 

    Args:
        extracted: ordered pairs of nodes representing the desired edges
        edges: all available edges in the graph

    Returns:
        paths: All feasable paths with the same node tuple sequence as in extracted
    """
    
    # Create a new directed graph
    path_graph= nx.DiGraph()

    # Iterate through all extracted paths
    for start,end in extracted:
        # Get all edges starting from the current node
        connections = edges[edges["start_stop_id"]==start]
        # Initialize an arbitrary large number
        i=1000000000

        # Iterate through all connections
        for index,row in connections.iterrows(): 
            # If the connection is not a walking path
            if row['is_walking']==0 :
                # Add the connection as a node to the graph
                add_node_to_graph(path_graph, index, row)
            else:
                # Add walking nodes to the graph
                i = add_walking_nodes_to_graph(path_graph, start, end, extracted, edges, row, i)


    # Iterate over all nodes in the graph
    for node_1_i, node_1 in path_graph.nodes(data=True):
        for node_2_i, node_2  in path_graph.nodes(data=True):
            # Skip if it's the same node
            if node_1_i == node_2_i:
                continue
            # If the end of the first node is the start of the second, and they are close in time
            if (node_1['end_stop_id']==node_2['start_stop_id'] and ((node_1['end_time']<= node_2['start_time'])or (abs(node_1['end_time'] - node_2['start_time']) <10) )):
                # Add a directed edge from the first node to the second with the travel time as the weight
                path_graph.add_edge(node_1_i, node_2_i, expected_travel_time= (node_2['start_time'] - node_1['start_time']))

    #This allows the route to start from any time 
    # Get all nodes that start from the first stop in the extracted path
    connections_from_start = [(node,attr) for node, attr in path_graph.nodes(data=True) if attr["start_stop_id"]==extracted[0][0]]
    # If no such nodes exist, return None
    if len(connections_from_start) == 0:
        return 
    # For all nodes that start from the first stop, add an edge from a "start" node to them with weight 0
    for node,attr in connections_from_start:
        path_graph.add_edge("start",node,expected_travel_time=0)
    
    # Get all nodes that end at the final stop in the extracted path
    connections_to_dest = [(node,attr) for node, attr in path_graph.nodes(data=True) if node!="start" and attr["end_stop_id"]==extracted[-1][1]]
    # If no such nodes exist, return None
    if len(connections_to_dest) == 0:
        return 
    # For all nodes that end at the final stop, add an edge from them to an "end" node with their expected travel time as the weight
    for node,attr in connections_to_dest:
        path_graph.add_edge(node,"end",expected_travel_time=attr["expected_travel_time"])

    # If there's a path from the "start" node to the "end" node
    if nx.has_path(path_graph,"start","end"):
        # Return the path with additional info added
        return add_path_info(path_graph)  
    else: 
        # If there's no path, return None
        return None


In [ ]:
from datetime import datetime

def calculate_total_time(data_list):
    """Calculate the total time between the start and end of a path

    Args:
        data_list: A list of dictionaries representing the path.

    Returns:
        total_time: The total time in seconds between the start and end
                    of the path.
    """
    time_format = '%H:%M:%S'

    # Convert the start time and end time to datetime objects
    start_time = datetime.strptime(data_list[0]['start_time'], time_format)
    end_time = datetime.strptime(data_list[-1]['end_time'], time_format)

    # Calculate the total time in seconds
    total_time = (end_time - start_time).total_seconds()

    return total_time

In [ ]:
def keep_k_fastest(paths, k):
    """Select the k fastest paths in a list

    Args:
        paths: list of list of dictionnary representing edges
        k: number of paths to keep

    Returns:
        shortest_paths: The k shortest paths
    """
    #Compute the time for each path
    paths_with_time = [(calculate_total_time(path), path) for path in paths]
    # Sort paths based on total time
    paths_with_time.sort(key=lambda x: x[0])  
    # Select the k shortest paths
    shortest_paths = paths_with_time[:k]  
    # Return only paths, without their total time
    shortest_paths = [path for total_time, path in shortest_paths]

    return shortest_paths


In [ ]:
def drop_duplucate_paths (paths):
    """Select the k fastest paths in a list

    Args:
        paths: list of list of dictionnary representing edges
        
    Returns:
        unique_paths: list of unique paths
        
    """
    unique_paths_dict = {}
    
    # A path is considered unique if it has a unique start time and end time
    for path in paths:
        start_time = path[0]['start_time']
        end_time = path[-1]['end_time']
        unique_paths_dict[(start_time, end_time)] = path

    # Get the unique paths from the dictionary
    unique_paths = list(unique_paths_dict.values())
    return unique_paths

In [ ]:
def compress_trips(trip_data):
    """Compresses a path so that each edge represents a unique 
        transportation mean

    Args:
        trip_data: list of dictionnary representing edges
        
    Returns:
        compressed_trips: A list of dictionnary where edges are compressed by trip_id
        
    """
    compressed_trips = []
    for trip in trip_data:
        # If the trip can not be compressed 
        if not compressed_trips or trip['trip_id'] != compressed_trips[-1]['trip_id'] or (trip['trip_id'] is None and compressed_trips[-1]['trip_id'] is None):
            compressed_trips.append(trip.copy())
        # If the trip can be compressed 
        else:
            compressed_trips[-1]['end_time'] = trip['end_time']
            compressed_trips[-1]['end_stop_id'] = trip['end_stop_id']
    return compressed_trips


In [ ]:
def get_k_paths(df_all_edges, source, sink, desired_arrival_time, max_trip_length, k):
    
    desired_arrival_time = get_sec(desired_arrival_time)
    # Filter the edges data frame based on whether the end time is within the range of the desired arrival time 
    # and the maximum trip length.
    df_all_edges = df_all_edges[(df_all_edges["is_walking"]==1) |  ((df_all_edges["end_time"] <= desired_arrival_time) & (df_all_edges["end_time"]>desired_arrival_time-max_trip_length))]
    
    # Create a directed graph using the filtered edges data frame.
    graph = nx.from_pandas_edgelist(df_all_edges, 'start_stop_id', 'end_stop_id', edge_attr=['expected_travel_time'],create_using=nx.DiGraph)
    
    # Generate a list of fastest paths (with a limit of 100) from the source to the sink node based on the expected travel time.
    fastest_paths = list(islice(nx.shortest_simple_paths(graph, source, sink, weight='expected_travel_time'), 100))
    
    result = []
    # For each path, zip together start and end stop IDs, create a data frame with these pairs and merge it with the filtered edges data frame.
    for path in fastest_paths:
        extracted = list(zip(path[:-1], path[1:]))
        df_pairs = pd.DataFrame(extracted, columns=["start_stop_id","end_stop_id"])
        df_edges_filtered = df_all_edges.merge(df_pairs, on=["start_stop_id","end_stop_id"], how="inner")
        
        # Get the path if it exists and append it to the result list.
        if get_path(extracted, df_edges_filtered):
            i = 0
            for p in get_path(extracted, df_edges_filtered):
                if i < 20:  # The maximum number of paths to be stored for each extracted path.
                    result.append(p)
                    i = i + 1
                    
    # Filter out the None values from the result list.
    paths= [r for r in result if r != None]
    
    # Drop duplicate paths and compress trips in each unique path.
    unique_paths = drop_duplucate_paths(paths)
    compressed_paths = [compress_trips(trip) for trip in unique_paths]
    
    # Get the fastest k paths from the compressed paths.
    fastest_k_paths = keep_k_fastest(compressed_paths, k)
    
    return fastest_k_paths


# Delay Estimation

We can now make use of our previous work for delay estimation. <br>
We start by importing df_mean which contains the average delay for each stop at each hour.
This will be really usefull in order to estimate the delays with exponential low!

In [ ]:
#import the table from hdfs
isdt = spark.read.orc('/group/grande_envergure/graph/isdt_delay_mean.orc')

#transform it to a pandas df
df_mean = isdt.toPandas()
df_mean.sample(5)

In [ ]:
isdt_delay = spark.read.orc('/group/grande_envergure/graph/isdt_delay.orc')
isdt_delay.show(5)

In [ ]:
pip install nbimporter

Then we import all methods from probabilistic_delay notebook

In [ ]:
import nbimporter
import probabilistic_delay 

# Grphical Interface 

In [ ]:
from IPython.display import display
from ipywidgets import widgets

We start by adding Widgets for parameter selection

In [ ]:
style = {'description_width': 'initial'}

number_routes = widgets.IntSlider(
    value=5,
    min=1,
    max=10,
    step=1,
    description='Number of Paths',
    style=style
)

max_trip_length = widgets.IntSlider(
    value=2,
    min=1,
    max=3,
    step=1,
    description='Max trip duration (H)',
    style=style
)
hours = widgets.BoundedFloatText(min=0, max=23, value=12, step=1, description='Hour')
minutes = widgets.BoundedFloatText(min=0, max=59, value=0, step=1, description='Minute')
validate_butt = widgets.ToggleButton(
    value=False,
    description='Show validation',
    style=style,
    tooltip='Description',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

stop_names = sorted(nodes_zurich['stop_name'].unique())

start = widgets.Dropdown(options=stop_names, description='From:')
start.value ='Egg'

end = widgets.Dropdown(options=stop_names, description='To:')
end.value = 'Zürich Flughafen'

button = widgets.Button(description="Run query")
data_output = widgets.Output()
map_output = widgets.Output()

We then define methods to handle display of paths as dataframes and maps 

In [ ]:
def create_map(df):
    # Extract start and end stops into separate dataframes
    start_stops = df[['start_stop_name', 'start_lat', 'start_lon']]
    end_stops = df[['end_stop_name', 'end_lat', 'end_lon']]

    # Rename columns to uniform names
    start_stops.columns = ['stop_name', 'stop_lat', 'stop_lon']
    end_stops.columns = ['stop_name', 'stop_lat', 'stop_lon']

    # Concatenate the start and end dataframes into a single dataframe
    map_df = pd.concat([start_stops, end_stops])

    fig = go.Figure()  # Initialize the figure

    # add the lines
    for _, row in df.iterrows():
        fig.add_trace(
            go.Scattermapbox(
                mode = "lines",
                lon = [row['start_lon'], row['end_lon']],
                lat = [row['start_lat'], row['end_lat']],
                marker = {'size': 10},
                hoverinfo = 'none', ))

    # Add the nodes after the lines
    fig.add_trace(
        go.Scattermapbox(
            lat=map_df["stop_lat"],
            lon=map_df["stop_lon"],
            mode='markers',
            marker=dict(size=10, color='red'),  # changed color to red
            text=map_df["stop_name"],
            hoverinfo='text'))

    fig.update_layout(
        mapbox_style="open-street-map",
        hovermode='closest',
        mapbox=dict(
            bearing=0,
            center=dict(
                lat=47.3769, 
                lon=8.5417
            ),
            pitch=0,
            zoom=10
        ),
    )
    
    return fig


def print_data(df):
    data_output.clear_output()
    with data_output :
        display(df)
    
def print_map(df):
    map_output.clear_output()
    with map_output :
        display(create_map(df))

In [ ]:
# Function triggered when a button is clicked
def on_button_clicked(btn):
    
    # Retrieve all the parameters from the widget values
    k = int(number_routes.value)
    source = start.value
    source = nodes_zurich[nodes_zurich['stop_name']== source]['stop_id'].values[0]
    sink = end.value
    sink = nodes_zurich[nodes_zurich['stop_name']== sink]['stop_id'].values[0]
    time = str(int(hours.value)).zfill(2) + ":"+ str(int(minutes.value)).zfill(2) + ":00" 
    max_length = int(max_trip_length.value)
    validate = validate_butt.value
    
    # Call the function to get the top k paths with provided parameters
    paths = get_k_paths(df_all_edges= df_all_edges, source=source,sink=sink,desired_arrival_time= time,max_trip_length=max_length * 60 * 60 ,k=k)
    paths.sort(key = lambda path: path[0]["start_time"], reverse=True)
    
    paths_proba = [probabilistic_delay.trip_conf(path,df_mean) for path in paths]
    if validate:
        paths_validate=[probabilistic_delay.trip_val(path,isdt_delay) for path in paths]
    # Convert each path into a pandas DataFrame and store them into a list
    dfs = [pd.DataFrame(path) for path in paths]
    

    # Create a tab widget for displaying the DataFrame of each path
    tab = widgets.Tab()
    data_output_tabs = [widgets.Output() for _ in dfs]
    tab.children = data_output_tabs

    # Create a tab widget for displaying the map of each path
    map_tab = widgets.Tab()
    map_output_tabs = [widgets.Output() for _ in dfs]
    map_tab.children = map_output_tabs
    
    # Check if the list is not empty
    if dfs:  
        for i, df in enumerate(dfs):
            # Set title for each tab
            tab.set_title(i, f'Path {i + 1}')
            # Fill in extra information about stations 
            df['start_stop_name'] = df.apply(lambda row: nodes_zurich[nodes_zurich['stop_id'] == row['start_stop_id']]['stop_name'].values[0], axis=1)
            df['end_stop_name'] = df.apply(lambda row: nodes_zurich[nodes_zurich['stop_id'] == row['end_stop_id']]['stop_name'].values[0], axis=1)
            df['walking'] = df.apply(lambda row: row['trip_id'] == 'None', axis=1)
            df['start_lat'] = df['start_stop_id'].apply(lambda id: nodes_zurich[nodes_zurich['stop_id'] == id]['stop_lat'].values[0])
            df['start_lon'] = df['start_stop_id'].apply(lambda id: nodes_zurich[nodes_zurich['stop_id'] == id]['stop_lon'].values[0])
            df['end_lat'] = df['end_stop_id'].apply(lambda id: nodes_zurich[nodes_zurich['stop_id'] == id]['stop_lat'].values[0])
            df['end_lon'] = df['end_stop_id'].apply(lambda id: nodes_zurich[nodes_zurich['stop_id'] == id]['stop_lon'].values[0])
            
           # Create greeting label and a DataFrame widget
            if validate :
                greeting = widgets.Label(f'The probability to be able to achieve path {i + 1} is {int(paths_proba[i]*100)}% and {int(paths_validate[i]*100)}% of identical paths succeeded in historical data')
            else :
                greeting = widgets.Label(f'The probability to be able to achieve path {i + 1} is {int(paths_proba[i]*100)}%')
            # Display VBox in corresponding output widget
            with data_output_tabs[i]:
                data_output_tabs[i].clear_output()
                display(greeting)  # Display greeting label
                display(df[['start_stop_name', 'end_stop_name', 'start_time', 'end_time', 'walking']])
            
            # Set title for each map tab
            map_tab.set_title(i, f'Map {i + 1}')
            
            # Display map in corresponding output widget
            with map_output_tabs[i]:   
                map_output_tabs[i].clear_output()
                display(create_map(df))

        # Clear old outputs and display the tab widget in data_output widget
        with data_output:
            data_output.clear_output()
            display(tab)
        
        # Clear old outputs and display the tab widget in map_output widget
        with map_output:
            map_output.clear_output()
            display(map_tab)
    else:
        # If no paths, print an empty DataFrame and an empty map
        print_data(pd.DataFrame())  
        print_map(pd.DataFrame()) 
        
button.on_click(on_button_clicked)

Finally, we display all the widgets together 

# Normal querries take around 15 seconds but reaches 2 min if validation is activated

In [ ]:
input_widgets = widgets.HBox([hours, minutes, number_routes,validate_butt])
nodes_widgets = widgets.HBox([start, end, max_trip_length])

all_widgets = widgets.VBox([input_widgets, nodes_widgets, button])

tab = widgets.Tab([data_output, map_output])
tab.set_title(0, 'Planning')
tab.set_title(1, 'Map')

dashboard = widgets.VBox([all_widgets, tab])
display(dashboard)